In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:90% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:2px;}
div.CodeMirror {font-family:Consolas; font-size:10pt;}
div.text_cell_render.rendered_html{font-size:10pt;}
div.output {font-size:10pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:10pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:11pt;padding:4px;}
table.dataframe{font-size:10px;}
</style>
"""))

# 1. 데이터 가져오기

In [1]:
import pandas as pd
df = pd.read_csv('c:/AI_X/download/shareData/부동산_250213/최종 전국 평균 평당 분양가격(결측치제외).csv', encoding='cp949')
df.head()

,지역명,연도,월,평당분양가격
0,서울,2013,12,18189.0
1,부산,2013,12,8111.0
2,대구,2013,12,8080.0
3,인천,2013,12,10204.0
4,광주,2013,12,6098.0


# 2. 지역명의 라벨 인코딩
- 지역명을 라벨 인코딩한 지역명2
- 분석할 경우 원핫인코딩까지 할 것을 추천

In [4]:
from tensorflow.keras.utils import to_categorical  # 분류분석시 원한 인코딩
from tensorflow.keras.models import Sequential   # 모델 생성
from tensorflow.keras.layers import Dense,Input
import numpy as np
import pandas as pd

In [3]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['지역명2'] = le.fit_transform(df['지역명'])
df['지역명2']

0        8
1        7
2        5
3       11
4        4
        ..
2171    13
2172    12
2173     3
2174     2
2175    14
Name: 지역명2, Length: 2176, dtype: int32

In [7]:
df.sample()
df['지역명2'].value_counts()

8     128
0     128
2     128
3     128
12    128
13    128
15    128
16    128
9     128
7     128
1     128
10    128
6     128
4     128
11    128
5     128
14    128
Name: 지역명2, dtype: int64

# 3. normalization 스케일 조정
- 입력변수(지역명2, 연도, 월)와 타겟변수(평당분양가격) 따로 스케일 조정(MinMaxScaler이용)
- 지역명2n, 연도n, 월n, 평당분양가격n 추가

In [13]:
from sklearn.preprocessing import MinMaxScaler
x_data = df[['지역명2','연도','월']].values
y_data = df[['평당분양가격']].values
x_data.shape, y_data.shape
x_scaler = MinMaxScaler()
y_scaler = MinMaxScaler()
df[['지역명2n','연도n','월n']] = x_scaler.fit_transform(x_data)
df['평당분양가격n'] = y_scaler.fit_transform(y_data)
df.head()

,지역명,연도,월,평당분양가격,지역명2,지역명2n,연도n,월n,평당분양가격n
0,서울,2013,12,18189.0,8,0.5000,0.0,1.0,0.328198
1,부산,2013,12,8111.0,7,0.4375,0.0,1.0,0.065274
2,대구,2013,12,8080.0,5,0.3125,0.0,1.0,0.064466
3,인천,2013,12,10204.0,11,0.6875,0.0,1.0,0.119878
4,광주,2013,12,6098.0,4,0.2500,0.0,1.0,0.012757


# 4. standarization 스케일 조정
- 입력변수와 타겟변수 따로 스케일 조정(StandardScaler이용)
- 지역명2s, 연도s, 월s, 평당분양가격s필드 추가

In [14]:
from sklearn.preprocessing import StandardScaler
x_scaler = StandardScaler()
y_scaler = StandardScaler()
df[['지역명2s','연도s','월s']] = x_scaler.fit_transform(x_data)
df['평당분양가격s'] = y_scaler.fit_transform(y_data)
df.head()

,지역명,연도,월,평당분양가격,지역명2,지역명2n,연도n,월n,평당분양가격n,지역명2s,연도s,월s,평당분양가격s
0,서울,2013,12,18189.0,8,0.5000,0.0,1.0,0.328198,0.000000,-1.875367,1.62196,1.168591
1,부산,2013,12,8111.0,7,0.4375,0.0,1.0,0.065274,-0.204124,-1.875367,1.62196,-0.728312
2,대구,2013,12,8080.0,5,0.3125,0.0,1.0,0.064466,-0.612372,-1.875367,1.62196,-0.734147
3,인천,2013,12,10204.0,11,0.6875,0.0,1.0,0.119878,0.612372,-1.875367,1.62196,-0.334363
4,광주,2013,12,6098.0,4,0.2500,0.0,1.0,0.012757,-0.816497,-1.875367,1.62196,-1.107203


# 5. 지역명을 원핫인코딩

In [15]:
from tensorflow.keras.utils import to_categorical
to_categorical(df['지역명2'])

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.]], dtype=float32)

In [24]:
# 지역명, 지역명2
loc_info = df[['지역명','지역명2']].head(17).sort_values(by='지역명2')
loc_column_name = loc_info['지역명'].tolist()

In [26]:
import numpy as np
df[loc_column_name] = to_categorical(df['지역명2']).astype(np.int16)
df.head()

,지역명,연도,월,평당분양가격,지역명2,지역명2n,연도n,월n,평당분양가격n,지역명2s,...,부산,서울,세종,울산,인천,전남,전북,제주,충남,충북
0,서울,2013,12,18189.0,8,0.5000,0.0,1.0,0.328198,0.000000,...,0,1,0,0,0,0,0,0,0,0
1,부산,2013,12,8111.0,7,0.4375,0.0,1.0,0.065274,-0.204124,...,1,0,0,0,0,0,0,0,0,0
2,대구,2013,12,8080.0,5,0.3125,0.0,1.0,0.064466,-0.612372,...,0,0,0,0,0,0,0,0,0,0
3,인천,2013,12,10204.0,11,0.6875,0.0,1.0,0.119878,0.612372,...,0,0,0,0,1,0,0,0,0,0
4,광주,2013,12,6098.0,4,0.2500,0.0,1.0,0.012757,-0.816497,...,0,0,0,0,0,0,0,0,0,0


In [28]:
pd.options.display.max_columns = 30  # 최대 출력다능한 데이터프레임 열수
df.head()

,지역명,연도,월,평당분양가격,지역명2,지역명2n,연도n,월n,평당분양가격n,지역명2s,연도s,월s,평당분양가격s,강원,경기,경남,경북,광주,대구,대전,부산,서울,세종,울산,인천,전남,전북,제주,충남,충북
0,서울,2013,12,18189.0,8,0.5000,0.0,1.0,0.328198,0.000000,-1.875367,1.62196,1.168591,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,부산,2013,12,8111.0,7,0.4375,0.0,1.0,0.065274,-0.204124,-1.875367,1.62196,-0.728312,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,대구,2013,12,8080.0,5,0.3125,0.0,1.0,0.064466,-0.612372,-1.875367,1.62196,-0.734147,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,인천,2013,12,10204.0,11,0.6875,0.0,1.0,0.119878,0.612372,-1.875367,1.62196,-0.334363,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4,광주,2013,12,6098.0,4,0.2500,0.0,1.0,0.012757,-0.816497,-1.875367,1.62196,-1.107203,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
